In [ ]:
! "c:\DATA SCIENCE\GEN AI\LangChain\Langchain Models\venv\Scripts\python.exe" -m pip install sentence-transformers



[notice] A new release of pip is available: 23.2.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [77]:
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from dotenv import load_dotenv
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts import PromptTemplate

In [78]:
load_dotenv()

# Initialize LLM
model = ChatGroq(
    model="meta-llama/llama-4-scout-17b-16e-instruct",
    temperature=0.5,
    max_tokens=512,
)


In [79]:
url = f"https://python.langchain.com/docs/integrations/tools/"

loader = WebBaseLoader(url)

document = loader.load()

In [80]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300,
    chunk_overlap = 50,
)

docs = splitter.split_documents(document)

In [81]:
# Initialize embeddings
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

# Create FAISS vector store from documents
vector_store = FAISS.from_documents(docs, embedding=embeddings)

In [82]:
retriever = vector_store.as_retriever(
    search_type="mmr",                   # <-- This enables MMR
    search_kwargs={"k": 3, "lambda_mult": 0.5}  # k = top results, lambda_mult = relevance-diversity balance
)

In [83]:
qa_prompt = PromptTemplate(
    template="""
You are a helpful assistant that answers questions based on a webpage.

Conversation so far:
{history}

Here is the webpage text (retrieved context):
----------------
{context}
----------------

Now, using ONLY the webpage text above:
Q: {question}
A:""",
    input_variables=["history", "context", "question"]
)



# Maintain chat history
messages = []

# Chat loop
while True:
    query = input("You: ")
    if query.lower() == "exit":
        print("Thank you, see you soon 👋")
        break
    print(f"You: {query}")
    # Append user message
    messages.append({"role": "user", "content": query})

    # Build history context
    history_context = "\n".join(
        [f"{m['role'].capitalize()}: {m['content']}" for m in messages]
    )

    # Retrieve relevant docs from vector DB (wiki/web)
    results = retriever.invoke(query)
    docs_context = "\n".join([doc.page_content for doc in results])

    # Final context (history + docs)
    formatted_prompt = qa_prompt.format(
    history=history_context,
    context=docs_context if docs_context.strip() else "No relevant text found.",
    question=query
    )

    # Call model directly (LLM)
    ans = model.invoke(formatted_prompt)

    # ✅ Fix: extract content properly
    response = ans.content

    # Append AI message
    messages.append({"role": "assistant", "content": response})

    print(f"AI: {response}")


You: 
AI: I'm ready to help. What is your question? 

(Please go ahead and ask your question, and I'll answer based on the webpage text provided)
Thank you, see you soon 👋
